In [1]:
!pip install pymystem3


Defaulting to user installation because normal site-packages is not writeable


# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**
   * **children** — количество детей в семье (int64)
   * **days_employed** — общий трудовой стаж в днях (float64)
   * **dob_years** — возраст клиента в годах (int64)
   * **education** — уровень образования клиента (object)
   * **education_id** — идентификатор уровня образования (int64)
   * **family_status** — семейное положение (object)
   * **family_status_id** — идентификатор семейного положения (int64)
   * **gender** — пол клиента (object)
   * **income_type** — тип занятости (object)
   * **debt** — имел ли задолженность по возврату кредитов (int64)
   * **total_income** — ежемесячный доход (float64)
   * **purpose** — цель получения кредита (object)


# 1. Загрузка и исследование данных 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
 

In [3]:
df = pd.read_csv(r'C:\Users\Leonid\Desktop\01_reliability_of_borrowers\data.csv')

In [4]:
df.head(3)

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21525 non-null  int64  
 1   children          21525 non-null  int64  
 2   days_employed     19351 non-null  float64
 3   dob_years         21525 non-null  int64  
 4   education         21525 non-null  object 
 5   education_id      21525 non-null  int64  
 6   family_status     21525 non-null  object 
 7   family_status_id  21525 non-null  int64  
 8   gender            21525 non-null  object 
 9   income_type       21525 non-null  object 
 10  debt              21525 non-null  int64  
 11  total_income      19351 non-null  float64
 12  purpose           21525 non-null  object 
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


In [6]:
df.shape

(21525, 13)

In [7]:
df.duplicated().sum()

0

In [8]:
df.index

RangeIndex(start=0, stop=21525, step=1)

In [9]:
df.describe()

,Unnamed: 0,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,10762.000000,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,6213.876608,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,5381.000000,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,10762.000000,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,16143.000000,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,21524.000000,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


# №2 Предобработка данных

In [10]:
df.reset_index(drop=True) # ресетнули индекс перед удалением ненужного столбца

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [11]:
df = df.drop('Unnamed: 0', axis=1) # удалили ненужный столбец

In [12]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [13]:
df.columns 

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [14]:
colms = ['number_of_children',
        'days_employed',
        'client_age_in_years',
        'education_level',
        'education_level_id',
        'family_status',
        'family_status_id',
        'gender',
        'employment_type',
        'credit_repayment_status',
        'monthly_income',
        'credit_purpose'
       ]
df.set_axis(colms, axis='columns', inplace=True) # переименовали столбцы

In [15]:
df.head(5)

,number_of_children,days_employed,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [16]:
df['number_of_children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [17]:
df.isna().sum() # посмотрели где пустые строки 

number_of_children            0
days_employed              2174
client_age_in_years           0
education_level               0
education_level_id            0
family_status                 0
family_status_id              0
gender                        0
employment_type               0
credit_repayment_status       0
monthly_income             2174
credit_purpose                0
dtype: int64

In [18]:
df['days_employed'] = df['days_employed'].fillna(0) # заменили на 0

In [19]:
df['monthly_income'] = df['monthly_income'].fillna(0) # заменили на 0

In [20]:
df.isna().sum()

number_of_children         0
days_employed              0
client_age_in_years        0
education_level            0
education_level_id         0
family_status              0
family_status_id           0
gender                     0
employment_type            0
credit_repayment_status    0
monthly_income             0
credit_purpose             0
dtype: int64

In [21]:
df.duplicated().sum() # посмтотрели дубли

54

In [22]:
df = df.drop_duplicates().reset_index(drop=True) # удалили

In [23]:
df.duplicated().sum()

0

In [24]:
df.head(5)

,number_of_children,days_employed,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [25]:
df['number_of_children'].unique()  

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [26]:
df[df['number_of_children'] == -1]['number_of_children'].count() 

47

In [27]:
df[df['number_of_children'] == 20]['number_of_children'].count()

76

In [28]:
df['number_of_children'] = df['number_of_children'].replace(-1, 1)

In [29]:
df[df['number_of_children'] == -1]['number_of_children'].count()

0

In [30]:
df['number_of_children'] = df['number_of_children'].replace(20, 2)

In [31]:
df[df['number_of_children'] == -1]['number_of_children'].count()

0

In [32]:
df['number_of_children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [33]:
df['number_of_children'].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: number_of_children, dtype: int64

In [34]:
df.head(5)

,number_of_children,days_employed,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [35]:
df['days_employed'].describe()

count     21471.000000
mean      56821.423140
std      135010.270744
min      -18388.949901
25%       -2522.536607
50%        -989.271304
75%           0.000000
max      401755.400475
Name: days_employed, dtype: float64

In [36]:
df['days_employed'] = df['days_employed'].abs()

In [37]:
df['days_employed'].describe()

count     21471.000000
mean      60307.713617
std      133489.355354
min           0.000000
25%         620.736110
50%        1818.689386
75%        4794.911909
max      401755.400475
Name: days_employed, dtype: float64

In [38]:
df['years_employed'] = df['days_employed'] / 365

In [39]:
df['years_employed'].describe()

count    21471.000000
mean       165.226613
std        365.724261
min          0.000000
25%          1.700647
50%          4.982711
75%         13.136745
max       1100.699727
Name: years_employed, dtype: float64

In [40]:
del df['days_employed']

In [41]:
df.head(5)

,number_of_children,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.235814


In [42]:
df['client_age_in_years'].describe()

count    21471.000000
mean        43.279074
std         12.574291
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: client_age_in_years, dtype: float64

In [43]:
df['client_age_in_years'].value_counts()

35    616
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    497
27    493
52    484
56    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
59    443
55    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: client_age_in_years, dtype: int64

In [44]:
df.head(5)

,number_of_children,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.235814


In [45]:
df['education_level'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [46]:
df['education_level'] = df['education_level'].str.lower()

In [47]:
df['education_level'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [48]:
df['education_level_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [49]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [50]:
df['family_status'] = df['family_status'].str.lower()

In [51]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [52]:
df['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [53]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [54]:
df[df['gender'] == 'XNA']['gender'].count()

1

In [55]:
df[df['gender'] == 'XNA']

,number_of_children,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose,years_employed
10690,0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости,6.461919


In [56]:
df.drop(index=[10690],axis=0,inplace=True)

In [57]:
df[df['gender'] == 'XNA']

,number_of_children,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose,years_employed


In [58]:
df['employment_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [59]:
df['credit_repayment_status'].unique()

array([0, 1], dtype=int64)

In [60]:
df['monthly_income'].describe()

count    2.147000e+04
mean     1.508889e+05
std      1.097774e+05
min      0.000000e+00
25%      8.894660e+04
50%      1.357134e+05
75%      1.957505e+05
max      2.265604e+06
Name: monthly_income, dtype: float64

In [61]:
df['monthly_income'] = df['monthly_income'].astype(int)

In [62]:
df['monthly_income'].head(5)

0    253875
1    112080
2    145885
3    267628
4    158616
Name: monthly_income, dtype: int32

In [63]:
df['credit_purpose'].value_counts()


свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [64]:
from pymystem3 import Mystem

In [65]:
# m=Mystem()



In [66]:
# def lemmatization (credit_purpose):
    
#     lemma_tmp=m.lemmatize(credit_purpose)
              
#     if ('жилье' in lemma_tmp) or ('недвижимость' in lemma_tmp):
#         lemma_name_purpose='жилье'
#         return lemma_name_purpose
#     elif 'автомобиль' in lemma_tmp:
#         lemma_name_purpose='автомобиль'
#         return lemma_name_purpose
#     elif 'образование' in lemma_tmp:
#         lemma_name_purpose='образование'
#         return lemma_name_purpose
#     else:
#         lemma_name_purpose='личные цели'
#         return lemma_name_purpose

In [67]:
# purp_lem = df['credit_purpose'].apply(lemmatization) # Лемматизация прошла успешно, ОМГ 10 ЧАСОВ 

In [68]:
# purp_lem = df['credit_purpose']

In [69]:
df['credit_purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [70]:
df.head(5)

,number_of_children,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23.116912
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11.026860
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15.406637
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11.300677
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932.235814


In [71]:
df = df.drop('years_employed', axis=1)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21470 entries, 0 to 21470
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   number_of_children       21470 non-null  int64 
 1   client_age_in_years      21470 non-null  int64 
 2   education_level          21470 non-null  object
 3   education_level_id       21470 non-null  int64 
 4   family_status            21470 non-null  object
 5   family_status_id         21470 non-null  int64 
 6   gender                   21470 non-null  object
 7   employment_type          21470 non-null  object
 8   credit_repayment_status  21470 non-null  int64 
 9   monthly_income           21470 non-null  int32 
 10  credit_purpose           21470 non-null  object
dtypes: int32(1), int64(5), object(5)
memory usage: 1.9+ MB


# Вывод:
 + посмотрели строки где нет значений - сделали их 0, удалили дубли
 + прошлись по каждому столбцу и привели данные  к нужному формату


# Анализ данных

# Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [73]:
one = df.groupby('number_of_children').agg({'credit_repayment_status':['count','sum']})

In [74]:
one 


credit_repayment_status      
                                     count   sum
number_of_children                              
0                                    14106  1063
1                                     4856   445
2                                     2128   202
3                                      330    27
4                                       41     4
5                                        9     0

In [75]:
one['raznica'] = one[('credit_repayment_status',   'sum')] * 100 / one[('credit_repayment_status',   'count')]

In [76]:
one

credit_repayment_status         raznica
                                     count   sum          
number_of_children                                        
0                                    14106  1063  7.535800
1                                     4856   445  9.163921
2                                     2128   202  9.492481
3                                      330    27  8.181818
4                                       41     4  9.756098
5                                        9     0  0.000000

### Вывод: зависимость есть, чем больше детей - тем больше родители тратят денег и тем выше шанс просрочки по кредиту 

# Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [77]:
two = df.groupby('family_status').agg({'credit_repayment_status':['count','sum']})
two

credit_repayment_status     
                                        count  sum
family_status                                     
в разводе                                1195   85
вдовец / вдова                            959   63
гражданский брак                         4162  388
женат / замужем                         12344  931
не женат / не замужем                    2810  274

In [78]:
two['raznica'] = two[('credit_repayment_status',   'sum')] * 100 / two[('credit_repayment_status',   'count')]
two

credit_repayment_status        raznica
                                        count  sum          
family_status                                               
в разводе                                1195   85  7.112971
вдовец / вдова                            959   63  6.569343
гражданский брак                         4162  388  9.322441
женат / замужем                         12344  931  7.542126
не женат / не замужем                    2810  274  9.750890

### Вывод: 
 + низкий процент просрочки тех кто в разводе или вдовец/вдова обусловден высоким уровнем ответственности. 
 + высокий уровень просрочки у тех кто в граждаком браке или не женат/ не замужем обусловлен их не готовностью брать на себя отственность 
 + те кто женат/замужем имеют нормальный уровень просрочки по кредиту

# Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


In [79]:
df['monthly_income'].describe()

count    2.147000e+04
mean     1.508885e+05
std      1.097774e+05
min      0.000000e+00
25%      8.894575e+04
50%      1.357130e+05
75%      1.957502e+05
max      2.265604e+06
Name: monthly_income, dtype: float64

In [80]:
def total_category_income(income):
    if 20000<=income<70000:
        return "низкий доход"
    if 70001<=income<120000:
        return 'средний доход'
    if 120001<=income<200000:
        return 'высокий доход'
    return 'очень высокий доход'

In [81]:
b_category_income = df['monthly_income'].apply(total_category_income)
b_category_income

0        очень высокий доход
1              средний доход
2              высокий доход
3        очень высокий доход
4              высокий доход
                ...         
21466    очень высокий доход
21467          высокий доход
21468          средний доход
21469    очень высокий доход
21470          средний доход
Name: monthly_income, Length: 21470, dtype: object

In [82]:
df['category_income'] = b_category_income
df

,number_of_children,client_age_in_years,education_level,education_level_id,family_status,family_status_id,gender,employment_type,credit_repayment_status,monthly_income,credit_purpose,category_income
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,очень высокий доход
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний доход
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,высокий доход
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,очень высокий доход
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,высокий доход
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,очень высокий доход
21467,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,высокий доход
21468,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,средний доход
21469,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,очень высокий доход


In [83]:
three = df.groupby('category_income').agg({'credit_repayment_status':['count','sum']})
three

credit_repayment_status     
                                      count  sum
category_income                                 
высокий доход                          7440  662
низкий доход                           1474  101
очень высокий доход                    7185  528
средний доход                          5371  450

In [84]:
three['raznica'] = three[('credit_repayment_status',   'sum')] * 100 / three[('credit_repayment_status',   'count')]
three

credit_repayment_status        raznica
                                      count  sum          
category_income                                           
высокий доход                          7440  662  8.897849
низкий доход                           1474  101  6.852103
очень высокий доход                    7185  528  7.348643
средний доход                          5371  450  8.378328

### Вывод: очень странно получается, у людей  с низким доходом уровень просрочки ниже чем у людей с высоким доходом